In [64]:
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import aiohttp
import time
import json

In [30]:
import requests
from bs4 import BeautifulSoup

# Define headers to mimic a real browser
gameId = 401671810

def clean_text(html_content):
    if not html_content:
        return "recap not found"
    soup = BeautifulSoup(html_content, "html.parser")
    # Extract all paragraph text
    paragraphs = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
    # Join them into a single text block
    cleaned_text = "\n\n".join(paragraphs)
    return cleaned_text

def pull_page_info(gameId):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Referer": "https://www.espn.com/",
    }
    url = f"https://www.espn.com/nfl/recap?gameId={gameId}"
    # Send request with headers
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    recap = soup.select_one(".Story__Body")  # Adjust based on ESPN's structure'



    return(clean_text(str(recap)).replace("\n\n", " "))


In [ ]:
client_session = aiohttp.ClientSession()

async def fetch(client_session, url):
  async with client_session.get(url) as resp:
    text()

In [32]:
pull_page_info(gameId - 3)

'MIAMI GARDENS, Fla. -- — Buffalo fans cheered so loudly in Hard Rock Stadium that Bills coach Sean McDermott was asked if it was a home game. It wasn\'t. But Buffalo certainly gave fans who made the trip to South Florida plenty to celebrate Thursday night. James Cook caught a touchdown pass and ran for two scores and the Bills routed Miami 31-10, with Dolphins quarterback Tua Tagovailoa leaving early because of a concussion. Tagovailoa left in the third quarter after colliding with defensive back Damar Hamlin and hitting the back of his head against the turf. Tagovailoa was quickly ruled out with the concussion, casting doubt on his season after a history of dealing with head injuries. “Very sad. It’s the worst part of the game," Bills quarterback Josh Allen said. “Nothing but love and prayers to him and his family, and hopefully everyone is sending those because he\'s going to need them.” Coach Mike McDaniel did not specify if the Dolphins will place Tagovailoa on injured reserve, sa

In [9]:
from newspaper import Article


In [57]:
url = 'https://walterfootball.com/nflreview2022_03.php'
article = Article(url)
article.download()
article.parse()

print((article.text).replace("\n", " "))

  Browns 29, Steelers 17  Everyone believed this game would be a defensive grinder where both scoring units would have trouble moving the chains consistently because of the expected 25-mph winds. There may have been a belief that this contest would be a carbon copy of the Bills-Patriots affair on a Monday night last year. Instead, both offenses were incredibly efficient in the opening half, with the two teams combining for 402 net yards of offense by intermission. The quarterbacks were not affected by the wind at all, as both Jacoby Brissett and Mitchell Trubisky were delivering terrific passes throughout the first two quarters. By halftime, this game was on pace to see 54 points, well above the betting total of 38.    The second half felt like a completely different game. The quarterbacks weren’t as great, with the two signal-callers combining for just 15 completions, some of which occurred via Trubisky’s garbage time. Brissett wasn’t entirely to blame, as his receivers dropped two pa

WalterFootball  The most accurate mock draft site in the world. NFL Football. The draft, picks and fantasy.


In [102]:
nba_articles = pd.read_csv("/Users/ethan/Sports_llm/sports_llm/nba_espn_articles.csv")
nfl_articles = pd.read_csv("/Users/ethan/Sports_llm/sports_llm/nfl_espn_articles.csv")


all_articles = pd.concat([nba_articles, nfl_articles])
all_articles.to_csv("/Users/ethan/Sports_llm/sports_llm/all_articles.csv", index=False)


In [104]:
len(all_articles)

16276

In [86]:
def get_game_ids(start_year = 2012, end_year = 2023, limit = 150):
  base_url = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard"
  game_ids = []

  for year in range(start_year, end_year + 1):
    url = f"{base_url}?limit={limit}&dates={year}"
    response = requests.get(url)

    if response.status_code == 200:
      data = response.json()
      for event in data["events"]:
        game_ids.append(event["id"])
    else:
      print(f"Failed to get game ids for {year}")
    time.sleep(1)
  return game_ids


In [63]:
get_ids = get_game_ids(2011, 2024)

In [65]:
with open("nfl_game_ids.json", "w") as file:
    json.dump(get_ids, file, indent=4)

In [93]:
import os

file_path = "all_articles.csv"

# Get file size in bytes
file_size = os.path.getsize(file_path)

print(f"File size: {file_size} bytes")


File size: 52963190 bytes


In [90]:
import requests
import json
import os
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_game_ids(date_str, base_url, limit=100):
    """Fetch game IDs for a specific date."""
    url = f"{base_url}?limit={limit}&dates={date_str}"
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            return [event.get("id") for event in data.get("events", []) if event.get("id")]
        else:
            print(f"Failed to fetch data for {date_str}, Status Code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {date_str}: {e}")
    return []

def get_nba_game_ids_multithreaded(season_year, limit=100, max_threads=10):
    """Retrieve NBA game IDs for a specific season using multithreading."""
    base_url = "https://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard"
    game_ids = []

    # Define season start and end dates
    season_start = datetime(season_year - 1, 10, 22)  # Regular Season Start
    season_end = datetime(season_year, 4, 14)  # Regular Season End

    # Generate list of all dates in the season
    date_list = [season_start + timedelta(days=i) for i in range((season_end - season_start).days + 1)]
    date_list = [d.strftime("%Y%m%d") for d in date_list]

    with ThreadPoolExecutor(max_threads) as executor:
        future_to_date = {executor.submit(fetch_game_ids, date, base_url, limit): date for date in date_list}

        for future in as_completed(future_to_date):
            game_ids.extend(future.result())

    return game_ids

# Define JSON file for storing all seasons
json_filename = "nba_game_ids.json"

# Load existing data if the JSON file already exists
all_game_ids = []

# Loop through seasons 2013-2024 and fetch game IDs
for year in range(2013, 2024):  # 2024-25 season is ongoing but can be included
    season_key = f"{year-1}-{year}"

    print(f"Fetching game IDs for the {season_key} NBA season...")
    game_ids = get_nba_game_ids_multithreaded(year)

    all_game_ids.extend(game_ids)  # ✅ Add all game IDs to a single list

    print(f"✅ Collected {len(game_ids)} game IDs for the {season_key} season.")

# ✅ Dump to JSON only once at the end
with open(json_filename, "w") as file:
    json.dump(all_game_ids, file, indent=4)

print(f"🏀 All game IDs from 2013 to 2024 have been fetched and saved in {json_filename}!")


Fetching game IDs for the 2012-2013 NBA season...
✅ Collected 1234 game IDs for the 2012-2013 season.
Fetching game IDs for the 2013-2014 NBA season...
✅ Collected 1245 game IDs for the 2013-2014 season.
Fetching game IDs for the 2014-2015 NBA season...
✅ Collected 1240 game IDs for the 2014-2015 season.
Fetching game IDs for the 2015-2016 NBA season...
✅ Collected 1246 game IDs for the 2015-2016 season.
Fetching game IDs for the 2016-2017 NBA season...
✅ Collected 1234 game IDs for the 2016-2017 season.
Fetching game IDs for the 2017-2018 NBA season...
✅ Collected 1200 game IDs for the 2017-2018 season.
Fetching game IDs for the 2018-2019 NBA season...
✅ Collected 1201 game IDs for the 2018-2019 season.
Fetching game IDs for the 2019-2020 NBA season...
✅ Collected 970 game IDs for the 2019-2020 season.
Fetching game IDs for the 2020-2021 NBA season...
✅ Collected 903 game IDs for the 2020-2021 season.
Fetching game IDs for the 2021-2022 NBA season...
✅ Collected 1230 game IDs for the 

In [133]:
import requests

def get_all_leagues():
# ESPN API base URL
    sports_url = "http://sports.core.api.espn.com/v2/sports/"
    headers = {"User-Agent": "Mozilla/5.0"}

    # Fetch the list of sports
    response = requests.get(sports_url, headers=headers)
    if response.status_code == 200:
        sports_data = response.json()

        sports_list = sports_data.get("items", [])

        leagues_dict = {}

        # Iterate through each sport
        for sport in sports_list:
            sport_url = sport.get("$ref")
            if sport_url:
                sport_response = requests.get(sport_url, headers=headers)
                if sport_response.status_code == 200:
                    sport_details = sport_response.json()
                    sport_name = sport_details.get("name", "Unknown Sport").lower()
                    leagues_url = sport_details.get("leagues", {}).get("$ref")
                    if leagues_url:
                        leagues_response = requests.get(leagues_url, headers=headers)
                        if leagues_response.status_code == 200:
                            leagues_data = leagues_response.json()
                            leagues_list = leagues_data.get("items", [])
                            league_names = []
                            for league in leagues_list:
                                league_url = league.get("$ref")
                                if league_url:
                                    league_response = requests.get(league_url, headers=headers)
                                    if league_response.status_code == 200:
                                        league_details = league_response.json()
                                        #print(league_details)
                                        league_name = league_details.get("slug", "Unknown League")
                                        league_names.append(league_name)

                            leagues_dict[sport_name] = league_names

        # Print results
        return(leagues_dict)


In [134]:
all_leagues_dict = get_all_leagues()

In [135]:
all_leagues_dict

{'australian rules football': ['afl'],
 'baseball': ['caribbean-series',
  'college-baseball',
  'college-softball',
  'dominican-winter-league',
  'llb',
  'mexican-winter-league',
  'mlb',
  'olympics-baseball',
  'puerto-rican-winter-league',
  'venezuelan-winter-league',
  'world-baseball-classic'],
 'basketball': ['fiba',
  'mens-college-basketball',
  'mens-olympics-basketball',
  'nba',
  'nba-development',
  'nba-summer-golden-state',
  'nba-summer-las-vegas',
  'nba-summer-orlando',
  'nba-summer-sacramento',
  'nba-summer-utah',
  'nbl',
  'wnba',
  'womens-college-basketball',
  'womens-olympics-basketball'],
 'cricket': [],
 'field hockey': ['womens-college-field-hockey'],
 'football': ['cfl', 'college-football', 'nfl', 'ufl', 'xfl'],
 'golf': ['champions-tour',
  'eur',
  'liv',
  'lpga',
  'mens-olympics-golf',
  'ntw',
  'pga',
  'womens-olympics-golf'],
 'ice hockey': ['hockey-world-cup',
  'mens-college-hockey',
  'nhl',
  'olympics-mens-ice-hockey',
  'olympics-womens

In [174]:
import requests
import json
import os
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_game_ids(date_str, base_url, limit=100):
    """Fetch game IDs for a specific date."""
    url = f"{base_url}?limit={limit}&dates={date_str}"
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            return [event.get("id") for event in data.get("events", []) if event.get("id")]
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass
    return []

def get_sport_game_ids_multithreaded(sport, league, season_year, limit=100, max_threads=10):
    """Retrieve NBA game IDs for a specific season using multithreading."""
    base_url = f"https://site.api.espn.com/apis/site/v2/sports/{sport}/{league}/scoreboard"
    game_ids = []

    # Define season start and end dates
    season_start = datetime(season_year - 1, 1, 1)  # Regular Season Start
    season_end = datetime(season_year, 12, 31)  # Regular Season End

    # Generate list of all dates in the season
    date_list = [season_start + timedelta(days=i) for i in range((season_end - season_start).days + 1)]
    date_list = [d.strftime("%Y%m%d") for d in date_list]

    with ThreadPoolExecutor(max_threads) as executor:
        future_to_date = {executor.submit(fetch_game_ids, date, base_url, limit): date for date in date_list}

        for future in as_completed(future_to_date):
            game_ids.extend(future.result())

    return game_ids

# Define JSON file for storing all seasons

# Load existing data if the JSON file already exists
all_game_ids =  []

def get_sport_ids(sport, league):
    all_game_ids =  {}
    json_filename = f"{league}_game_ids.json"
    # Loop through seasons 2013-2024 and fetch game IDs
    for year in range(2013, 2024):  # 2024-25 season is ongoing but can be included
        season_key = f"{year-1}-{year}"

        print(f"Fetching game IDs for the {season_key} {sport} season...")
        game_ids = get_sport_game_ids_multithreaded(sport, league, year)

        if league not in all_game_ids:
            all_game_ids[league] = []
        all_game_ids[league].extend(game_ids)

        print(f"✅ Collected {len(game_ids)} game IDs for the {league} {season_key} season.")

    # ✅ Dump to JSON only once at the end
    with open(json_filename, "w") as file:
        json.dump(all_game_ids, file, indent=4)

    print(f"All game IDs from 2013 to 2024 have been fetched and saved in {json_filename}!")


In [170]:
all_leagues_dict

{'australian rules football': ['afl'],
 'baseball': ['caribbean-series',
  'college-baseball',
  'college-softball',
  'dominican-winter-league',
  'llb',
  'mexican-winter-league',
  'mlb',
  'olympics-baseball',
  'puerto-rican-winter-league',
  'venezuelan-winter-league',
  'world-baseball-classic'],
 'basketball': ['fiba',
  'mens-college-basketball',
  'mens-olympics-basketball',
  'nba',
  'nba-development',
  'nba-summer-golden-state',
  'nba-summer-las-vegas',
  'nba-summer-orlando',
  'nba-summer-sacramento',
  'nba-summer-utah',
  'nbl',
  'wnba',
  'womens-college-basketball',
  'womens-olympics-basketball'],
 'cricket': [],
 'field hockey': ['womens-college-field-hockey'],
 'football': ['cfl', 'college-football', 'nfl', 'ufl', 'xfl'],
 'golf': ['champions-tour',
  'eur',
  'liv',
  'lpga',
  'mens-olympics-golf',
  'ntw',
  'pga',
  'womens-olympics-golf'],
 'ice hockey': ['hockey-world-cup',
  'mens-college-hockey',
  'nhl',
  'olympics-mens-ice-hockey',
  'olympics-womens

In [168]:
filtered_sports = {
    sport: leagues for sport, leagues in all_leagues_dict.items()
    if sport in {"baseball", "basketball", "football", "ice hockey"}
}

In [172]:
get_sport_ids("football", "nfl")

Fetching game IDs for the 2012-2013 football season...


KeyboardInterrupt: 

In [173]:
for key, values in filtered_sports.items():
    for value in values:
        get_sport_ids(key, value)


Fetching game IDs for the 2012-2013 baseball season...
✅ Collected 25 game IDs for the caribbean-series 2012-2013 season.
Fetching game IDs for the 2013-2014 baseball season...
✅ Collected 26 game IDs for the caribbean-series 2013-2014 season.
Fetching game IDs for the 2014-2015 baseball season...
✅ Collected 26 game IDs for the caribbean-series 2014-2015 season.
Fetching game IDs for the 2015-2016 baseball season...
✅ Collected 26 game IDs for the caribbean-series 2015-2016 season.
Fetching game IDs for the 2016-2017 baseball season...
✅ Collected 26 game IDs for the caribbean-series 2016-2017 season.
Fetching game IDs for the 2017-2018 baseball season...
✅ Collected 26 game IDs for the caribbean-series 2017-2018 season.
Fetching game IDs for the 2018-2019 baseball season...
✅ Collected 26 game IDs for the caribbean-series 2018-2019 season.
Fetching game IDs for the 2019-2020 baseball season...
✅ Collected 31 game IDs for the caribbean-series 2019-2020 season.
Fetching game IDs for th

KeyboardInterrupt: 